In [1]:
import urllib.request
from tqdm.notebook import tqdm
import requests
import six
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
url = '''https://en.wikipedia.org/wiki/Battle_of_the_Golden_Spurs'''

In [3]:
url_rus = '''https://ru.wikipedia.org/wiki/%D0%91%D0%B8%D1%82%D0%B2%D0%B0_%D0%BF%D1%80%D0%B8_%D0%9A%D1%83%D1%80%D1%82%D1%80%D0%B5'''

In [4]:
responce = requests.get(url)

In [5]:
responce

<Response [200]>

In [6]:
soup = BeautifulSoup(responce.content, 'html.parser')

In [7]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Battle of the Golden Spurs - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"7067f260-2817-48b6-9f89-eebdde54b871","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Battle_of_the_Golden_Spurs","wgTitle":"Battle of the Golden Spurs","wgCurRevisionId":1033248900,"wgRevisionId":1033248900,"wgArticleId":360246,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Harv and Sfn no-target errors","CS1 Dutch-language sources (nl)","Articles with short description","Short descr

In [8]:
soup.find_all('a')[6].text
# soup.find_all('a')[6].text[0].isupper()

'Grandes Chroniques de France'

In [9]:
ref_texts = [ref.text for ref in soup.find_all('a') if len(ref.text) > 0]
ref_texts = [ref.strip().replace(' ', '%20') for ref in ref_texts if ref[0].isupper()]
for i in range(len(ref_texts) - 1, -1, -1):
    if ref_texts.count(ref_texts[i]) > 1:
        ref_texts.pop(i)
    # else:
        # ref_texts[i] = ref_texts[i].strip().replace(' ', '_')
# May be set, may be list

In [10]:
ref_texts

['Jump%20to%20navigation',
 'Jump%20to%20search',
 'Battle%20of%20the%20Spurs',
 'Franco-Flemish%20War',
 'Grandes%20Chroniques%20de%20France',
 'Kortrijk',
 'County%20of%20Flanders',
 'Coordinates',
 'County%20of%20Namur',
 'Kingdom%20of%20France',
 'William%20of%20Jülich',
 'Guy%20of%20Namur',
 'John%20I%20of%20Namur',
 'Pieter%20de%20Coninck',
 'Jan%20Borluut',
 'Jan%20van%20Renesse',
 "Robert%20d'Artois",
 'John%20I,%20Count%20of%20Dammartin',
 'Belgium',
 'Furnes',
 'Bruges%20Matins',
 'Golden%20Spurs',
 'Arques',
 'Zierikzee',
 'Mons-en-Pévèle',
 'Flemish',
 'French',
 'France',
 'Bruges',
 'King%20Philip%20IV%20of%20France',
 'Count%20Robert%20II%20of%20Artois',
 'Flemish%20Movement',
 'Philip%20IV%20"the%20Fair"',
 'John%20of%20Avesnes',
 'Hainaut',
 'Holland',
 'Zeeland',
 'Guy%20of%20Dampierre',
 'English',
 'Courtrai%20Chest',
 'Edward%20I',
 'Jacques%20de%20Châtillon',
 'John',
 'Ghent',
 'West%20Flanders',
 'Ypres',
 'Annals%20of%20Ghent',
 'Leie',
 'Florentine',
 'Nuova%2

In [11]:
with open('mapbox_key.txt') as key:
    token = key.read()
    
searches = []
    
for place in ref_texts:
    search = f'''https://api.mapbox.com/geocoding/v5/mapbox.places/{place}.json?access_token={token}'''
    searches.append(search)

In [12]:
print(searches[18])

https://api.mapbox.com/geocoding/v5/mapbox.places/Belgium.json?access_token=pk.eyJ1IjoibWFya2l6enoiLCJhIjoiY2tyMXZwOTM1MXFoZjJucGxrejdrODlqciJ9.bCcnS9U0a_yw3jSftTIreA


In [13]:
answer = requests.get(searches[18])
# answer.json()['features'][0]['text']
answer.json()

{'type': 'FeatureCollection',
 'query': ['belgium'],
 'features': [{'id': 'country.1494593798110250',
   'type': 'Feature',
   'place_type': ['country'],
   'relevance': 1,
   'properties': {'wikidata': 'Q31', 'short_code': 'be'},
   'text': 'Belgium',
   'place_name': 'Belgium',
   'bbox': [2.51026658202508,
    49.4969670113735,
    6.40815299714662,
    51.5051159806493],
   'center': [4.63357500000001, 50.438696],
   'geometry': {'type': 'Point',
    'coordinates': [4.63357500000001, 50.438696]}},
  {'id': 'place.13106142554110250',
   'type': 'Feature',
   'place_type': ['place'],
   'relevance': 1,
   'properties': {'wikidata': 'Q2025327'},
   'text': 'Belgium',
   'place_name': 'Belgium, Wisconsin, United States',
   'bbox': [-87.9586959994111,
    43.4558380186788,
    -87.7911020436949,
    43.5434079355766],
   'center': [-87.8504, 43.4997],
   'geometry': {'type': 'Point', 'coordinates': [-87.8504, 43.4997]},
   'context': [{'id': 'district.15502135424015110',
     'wikidata

In [14]:
founded = []
STATUS_CODE_FOUND = 200

for i in tqdm(range(len(searches))):
    answer = requests.get(searches[i])
    if answer.status_code == STATUS_CODE_FOUND:
        # if answer.json()['query'] == ref_texts[i].replace(' ', '%20').lower():
        if len(answer.json()['features']) > 0: # Я здесь перебираю только первый результат запроса, улучши!
            if answer.json()['features'][0]['text'].lower().split() == ref_texts[i].lower().split('%20'):
                print(ref_texts[i].lower().split('%20'))
                print(answer.json()['features'][0]['center'])
                print()

['kortrijk']
[3.2659884, 50.8276429]

['belgium']
[4.63357500000001, 50.438696]

['furnes']
[11.0211, 60.8406]

['arques']
[2.3025, 50.73556]

['zierikzee']
[3.9164, 51.6497]

['mons-en-pévèle']
[3.10222, 50.47944]

['france']
[2.61878695312962, 47.8249046208979]

['hainaut']
[3.91667, 50.5]

['holland']
[-86.1091, 42.7876]

['zeeland']
[3.75, 51.56667]

['west', 'flanders']
[3, 51]

['breteuil']
[2.29333, 49.6325]

['vierzon']
[2.06833, 47.22194]

['aljubarrota']
[-8.9293, 39.5665]

['sempach']
[8.191823, 47.1347265]

['grandson']
[6.65, 46.81667]

['flanders']
[-74.6949, 40.8457]

['isbn']
[100.541078, 13.697264]

['suffolk']
[1, 52.1667]

['jstor']
[-83.745819, 42.279857]

['united', 'states']
[-97.9222112121185, 39.3812661305678]

['contact', 'us']
[-98.272474, 19.011998]

['ido']
[3.7119, 7.5068]

['latina']
[-3.786779, 40.387945]

['mobile', 'view']
[-80.5999285, 40.4760544]




In [17]:
import requests
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from typing import List, Dict, Any


class PageNotFound:
    def __init__(self):
        print('Page in not found. Please, check the URL.')


class WikiPlacesRecognition:

    # Code if any similar place is found
    STATUS_CODE_FOUND = 200

    FoundPlacesList = List[Dict[str, Any]]

    def __init__(self, api_token):
        self.api = api_token

    @staticmethod
    def from_api_token_path(token_file_path: str):
        with open(token_file_path) as api_token_file:
            api_token = api_token_file.read()
        return WikiPlacesRecognition(api_token=api_token)

    def __call__(self, wiki_page_url) -> FoundPlacesList:
        try:
            ref_texts = self._scrap_wiki_page(wiki_page_url)
        except PageNotFound:
            return []  # Error
        queries = []
        for place in ref_texts:
            search = f'https://api.mapbox.com/geocoding/v5/mapbox.places/{place}.json?access_token={self.api}'
            queries.append(search)
        founded_places = []
        for i in tqdm(range(len(queries))):
            answer = requests.get(queries[i])
            if answer.status_code == self.STATUS_CODE_FOUND:
                if len(answer.json()['features']) > 0:  # Я здесь перебираю только первый результат запроса, улучши!
                    if answer.json()['features'][0]['text'].lower().split() == ref_texts[i].lower().split('%20'):
                        founded_places.append({
                            'name': ' '.join(ref_texts[i].capitalize().split('%20')),
                            'longitude': answer.json()['features'][0]['center'][0],
                            'latitude': answer.json()['features'][0]['center'][1],
                        })
        return founded_places

    def _scrap_wiki_page(self, wiki_page_url) -> List[str]:
        response = requests.get(wiki_page_url)
        if response.status_code != self.STATUS_CODE_FOUND:
            raise PageNotFound
        soup = BeautifulSoup(response.content, 'html.parser')
        ref_texts = [ref.text for ref in soup.find_all('a') if len(ref.text) > 0]
        ref_texts = [ref.strip().replace(' ', '%20') for ref in ref_texts if ref[0].isupper()]
        for i in range(len(ref_texts) - 1, -1, -1):
            if ref_texts.count(ref_texts[i]) > 1:
                ref_texts.pop(i)
        return ref_texts


user_init = WikiPlacesRecognition.from_api_token_path('mapbox_key.txt')
cur_url = input()
print(user_init(cur_url))

https://en.wikipedia.org/wiki/Battle_of_the_Golden_Spurs



[{'name': 'Kortrijk', 'longitude': 3.2659884, 'latitude': 50.8276429}, {'name': 'Belgium', 'longitude': 4.63357500000001, 'latitude': 50.438696}, {'name': 'Furnes', 'longitude': 11.0211, 'latitude': 60.8406}, {'name': 'Arques', 'longitude': 2.3025, 'latitude': 50.73556}, {'name': 'Zierikzee', 'longitude': 3.9164, 'latitude': 51.6497}, {'name': 'Mons-en-pévèle', 'longitude': 3.10222, 'latitude': 50.47944}, {'name': 'France', 'longitude': 2.61878695312962, 'latitude': 47.8249046208979}, {'name': 'Hainaut', 'longitude': 3.91667, 'latitude': 50.5}, {'name': 'Holland', 'longitude': -86.1091, 'latitude': 42.7876}, {'name': 'Zeeland', 'longitude': 3.75, 'latitude': 51.56667}, {'name': 'West flanders', 'longitude': 3, 'latitude': 51}, {'name': 'Breteuil', 'longitude': 2.29333, 'latitude': 49.6325}, {'name': 'Vierzon', 'longitude': 2.06833, 'latitude': 47.22194}, {'name': 'Aljubarrota', 'longitude': -8.9293, 'latitude': 39.5665}, {'name': 'Sempach', 'longitude': 8.191823, 'latitude': 47.134726

In [27]:
import requests
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from typing import List, Dict, Any


class PageNotFound(BaseException):
    def __init__(self):
        print('Page in not found. Please, check the URL.')


class WikiPlacesRecognition:

    # Code if any similar place is found
    STATUS_CODE_FOUND = 200

    FoundPlacesList = List[Dict[str, Any]]

    def __init__(self, api_token):
        self.api = api_token

    @staticmethod
    def from_api_token_path(token_file_path: str):
        with open(token_file_path) as api_token_file:
            api_token = api_token_file.read()
        return WikiPlacesRecognition(api_token=api_token)

    def __call__(self, wiki_page_url) -> FoundPlacesList:
        try:
            ref_texts = self._scrap_wiki_page(wiki_page_url)
        except PageNotFound:
            return []  # Error
        queries = []
        for place in ref_texts:
            search = f'https://api.mapbox.com/geocoding/v5/mapbox.places/{place}.json?access_token={self.api}'
            queries.append(search)
        founded_places = []
        for i in tqdm(range(len(queries))):
            answer = requests.get(queries[i])
            if answer.status_code == self.STATUS_CODE_FOUND:
                if len(answer.json()['features']) > 0:
                    for variant in answer.json()['features']:
                        # print(variant['place_type'])
                        if variant['text'].lower().split() == ref_texts[i].lower().split('%20') and 'address' not in variant['place_type'] and 'poi' not in variant['place_type']:
                            founded_places.append({
                                'name': ' '.join(ref_texts[i].capitalize().split('%20')),
                                'longitude': variant['center'][0],
                                'latitude': variant['center'][1],
                            })
        return founded_places

    def _scrap_wiki_page(self, wiki_page_url) -> List[str]:
        response = requests.get(wiki_page_url)
        if response.status_code != self.STATUS_CODE_FOUND:
            raise PageNotFound
        soup = BeautifulSoup(response.content, 'html.parser')
        ref_texts = [ref.text for ref in soup.find_all('a') if len(ref.text) > 0]
        ref_texts = [ref.strip().replace(' ', '%20') for ref in ref_texts if ref[0].isupper()]
        for i in range(len(ref_texts) - 1, -1, -1):
            if ref_texts.count(ref_texts[i]) > 1:
                ref_texts.pop(i)
        return ref_texts


In [28]:
user_init = WikiPlacesRecognition.from_api_token_path('mapbox_key.txt')

In [29]:
print(user_init(url))


[{'name': 'Kortrijk', 'longitude': 3.2659884, 'latitude': 50.8276429}, {'name': 'Kortrijk', 'longitude': 4.9847, 'latitude': 52.1567}, {'name': 'Belgium', 'longitude': 4.63357500000001, 'latitude': 50.438696}, {'name': 'Belgium', 'longitude': -87.8504, 'latitude': 43.4997}, {'name': 'Furnes', 'longitude': 11.0211, 'latitude': 60.8406}, {'name': 'Arques', 'longitude': 2.3025, 'latitude': 50.73556}, {'name': 'Arques', 'longitude': 2.80056, 'latitude': 44.31694}, {'name': 'Arques', 'longitude': 2.375, 'latitude': 42.9525}, {'name': 'Zierikzee', 'longitude': 3.9164, 'latitude': 51.6497}, {'name': 'Mons-en-pévèle', 'longitude': 3.10222, 'latitude': 50.47944}, {'name': 'France', 'longitude': 2.61878695312962, 'latitude': 47.8249046208979}, {'name': 'Bruges', 'longitude': -0.6125, 'latitude': 44.88278}, {'name': 'Hainaut', 'longitude': 3.91667, 'latitude': 50.5}, {'name': 'Holland', 'longitude': -86.1091, 'latitude': 42.7876}, {'name': 'Holland', 'longitude': -83.7116, 'latitude': 41.6217}, 

In [31]:
requests.get(f'https://api.mapbox.com/geocoding/v5/mapbox.places/Holland.json?access_token=')

NameError: name 'self' is not defined